## Differentiable Greedy Algorithm for Monotone Submodular Maximization: Guarantees, Gradient Estimators, and Applications
This is an unofficial jupyter notebook for reproducing the experiments of influence maximization in [1].

[1] Shinsaku Sakaue: Differentiable Greedy Algorithm for Monotone Submodular Maximization: Guarantees, Gradient Estimators, and Applications, In. Proc. AISTATS, 2021.

### Import modules

In [1]:
%reload_ext autoreload
%autoreload 2
import torch
from src.dataset import MovieLensDataset
from src.model import NNModel, RandomModel
from torch.utils.data import DataLoader

### Define Hyperparameters for Dataset

In [2]:
N_INSTANCES = 120
N_MOVIES = 100
N_USERS = 500
RANDOM_STATE = 20250709
TRAIN_SIZE = 80
VALID_SIZE = 20
TEST_SIZE = 20
DATA_DIR = "./ml-100k-data"
DATA_ZIP_PATH = f"{DATA_DIR}/ml-100k.zip"
EXTRACTED_PATH = f"{DATA_DIR}/ml-100k"

### Prepare Dataset

In [3]:
dataset = MovieLensDataset(
    DATA_DIR,
    DATA_ZIP_PATH,
    n_instances=N_INSTANCES,
    n_movies=N_MOVIES,
    n_users=N_USERS,
    random_state=RANDOM_STATE,
)
print(dataset.theta.mean())

Dataset already exists.
Num Movies: 1682, Num Users: 943, Num Ratings:100000


100%|██████████| 120/120 [00:03<00:00, 30.38it/s]


tensor(0.0045)


### Define Hyperparameters for Training

In [ ]:
N_TRIALS = 5  # 30 in the original paper
EPOCHS = 5
BATCH_SIZE = 20
LR = 1e-3
OUTPUT = "clamp"
ACCELERATOR = "cuda" if torch.cuda.is_available() else "cpu"
N_PARTITION = 5  # for partition matroid constraint

### Define Functions for Training and Evaluating

In [ ]:
from collections import defaultdict

from lightning.pytorch import Trainer
from src.constraint import Cardinality, PartitionMatroid
from src.method import DFLContinuous, DFLSmoothedGreedy, TwoStage
from src.utility import InfluenceUtility

torch.use_deterministic_algorithms(True, warn_only=True)


def train_random(utility: InfluenceUtility, constraint: Cardinality | PartitionMatroid):
    """Prepare Random algorithm"""
    model = RandomModel()
    algorithm = TwoStage(
        model=model, utility=utility, constraint=constraint, selector="random"
    )
    trainer = Trainer(accelerator=ACCELERATOR, max_epochs=0)
    return algorithm, trainer


def train_twostage(
    dataset: MovieLensDataset,
    utility: InfluenceUtility,
    constraint: Cardinality | PartitionMatroid,
    **kwargs,
):
    """Train Two-stage model"""
    input_dim = dataset[0][0].shape[-1]
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    model = NNModel(input_dim, output=OUTPUT)
    algorithm = TwoStage(
        model=model,
        utility=utility,
        constraint=constraint,
        lr=LR,
        **kwargs,
    )
    trainer = Trainer(accelerator=ACCELERATOR, max_epochs=EPOCHS, inference_mode=False)
    trainer.fit(
        model=algorithm,
        train_dataloaders=train_loader,
    )
    return algorithm, trainer


def train_continuous(
    dataset: MovieLensDataset,
    utility: InfluenceUtility,
    constraint: Cardinality | PartitionMatroid,
    **kwargs,
):
    """Train Continuous model"""
    input_dim = dataset[0][0].shape[-1]
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    model = NNModel(input_dim, output=OUTPUT)
    algorithm = DFLContinuous(
        model=model,
        utility=utility,
        constraint=constraint,
        lr=LR,
        **kwargs,
    )
    trainer = Trainer(accelerator=ACCELERATOR, max_epochs=EPOCHS, inference_mode=False)
    trainer.fit(
        model=algorithm,
        train_dataloaders=train_loader,
    )
    return algorithm, trainer


def train_smoothed_greedy(
    dataset: MovieLensDataset,
    utility: InfluenceUtility,
    constraint: Cardinality | PartitionMatroid,
    **kwargs,
):
    """Train SG or VR-SG algorithm"""
    input_dim = dataset[0][0].shape[-1]
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    model = NNModel(input_dim, output=OUTPUT)
    print(kwargs)
    algorithm = DFLSmoothedGreedy(
        model=model,
        utility=utility,
        constraint=constraint,
        lr=LR,
        **kwargs,
    )
    trainer = Trainer(accelerator=ACCELERATOR, max_epochs=EPOCHS)
    trainer.fit(
        model=algorithm,
        train_dataloaders=train_loader,
    )
    return algorithm, trainer


def train_and_evaluate(
    dataset: MovieLensDataset,
    max_cardinality: int,
    methods_to_run: list[tuple[str, dict]],
    constraint: str = "cardinality",
):
    """
    Train and evaluate all algorithms on given instances with specified K (max_cardinality).
    """
    results = defaultdict(list)
    results_val = defaultdict(list)
    print(f"--- K = {max_cardinality} ---")
    utility = InfluenceUtility()

    if constraint == "cardinality":
        constraint = Cardinality(max_cardinality=max_cardinality)
    elif constraint == "partition":
        max_cardinalities = [max_cardinality] * N_PARTITION
        partition = [
            [j + i * (N_MOVIES // N_PARTITION) for j in range(N_MOVIES // N_PARTITION)]
            for i in range(N_PARTITION)
        ]
        constraint = PartitionMatroid(max_cardinalities, partition)

    torch.manual_seed(RANDOM_STATE)
    for _ in range(N_TRIALS):
        # split the dataset to training, valdating, and testing
        train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
            dataset, [TRAIN_SIZE, VALID_SIZE, TEST_SIZE]
        )
        val_loader = DataLoader(val_dataset, batch_size=TEST_SIZE, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=TEST_SIZE, shuffle=False)

        for method, kwargs in methods_to_run:
            if kwargs:
                kwargs_str = " " + ", ".join(
                    [f"{key}={value}" for key, value in kwargs.items()]
                )
            else:
                kwargs_str = ""
                kwargs = {}
            key = f"{method}{kwargs_str}"
            print(f"  Training: {key}")
            if method in ["Two-stage"]:
                algorithm, trainer = train_twostage(
                    train_dataset, utility, constraint, **kwargs
                )
            elif method in ["Continuous"]:
                algorithm, trainer = train_continuous(
                    train_dataset,
                    utility,
                    constraint,
                    **kwargs,
                )
            elif method in ["SG", "VR-SG"]:
                algorithm, trainer = train_smoothed_greedy(
                    train_dataset,
                    utility,
                    constraint,
                    variance_reduction=method == "VR-SG",
                    **kwargs,
                )
            elif method == "Random":
                algorithm, trainer = train_random(utility, constraint)

            print(f"  Evaluation: {key}")
            trainer.validate(model=algorithm, dataloaders=val_loader)
            results_val[key].append(trainer.logged_metrics["val_utility"].item())
            trainer.test(model=algorithm, dataloaders=test_loader)
            results[key].append(trainer.logged_metrics["test_utility"].item())
    return results, results_val

### Train, Evaluate, and Compares SG, VR-SG, Two-stage, and Random Under Cardinality Constraint

In [6]:
import logging
import warnings

warnings.filterwarnings("ignore")
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)

methods_to_run = [
    ("SG", {"n_sampling": 1}),
    ("SG", {"n_sampling": 10}),
    ("SG", {"n_sampling": 100}),
    ("VR-SG", {"n_sampling": 10}),
    ("VR-SG", {"n_sampling": 100}),
    ("Continuous", {"selector": "sga"}),
    ("Two-stage", {"selector": "greedy"}),
    ("Random", None),
]

constraint = "cardinality"
MAX_CARDINALITIES = [5, 10, 20]
full_results_table = {}
full_results_table_val = {}

for K in MAX_CARDINALITIES:
    results = train_and_evaluate(dataset, K, methods_to_run, constraint)
    full_results_table[K], full_results_table_val[K] = results

--- K = 5 ---
  Training: SG n_sampling=1
{'variance_reduction': False, 'n_sampling': 1}


2025-07-11 08:40:54.419781: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-11 08:40:54.433759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752223254.450559  449973 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752223254.455731  449973 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752223254.468737  449973 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Training: |          | 0/? [00:00<?, ?it/s]

  Evaluation: SG n_sampling=1


Validation: |          | 0/? [00:00<?, ?it/s]

...

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_utility          40.068416595458984
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  Training: Random
  Evaluation: Random


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       val_utility           39.38080978393555
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_utility           41.48175048828125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## Display the Results

In [7]:
import numpy as np
import pandas as pd

index = [key for key in full_results_table_val[MAX_CARDINALITIES[0]].keys()]
means = [
    [np.mean(val) for val in full_results_table_val[K].values()]
    for K in MAX_CARDINALITIES
]
stds = [
    [np.std(val) for val in full_results_table_val[K].values()]
    for K in MAX_CARDINALITIES
]
values = np.array(
    [
        [f"{m:.1f} ± {s:.1f}" for m, s in zip(means[i], stds[i])]
        for i in range(len(MAX_CARDINALITIES))
    ]
).T
print("Validation Results")
display(
    pd.DataFrame(
        values,
        index=index,
        columns=[
            f"test_utility (K={max_cardinality})"
            for max_cardinality in MAX_CARDINALITIES
        ],
    )
)

Validation Results


,test_utility (K=5),test_utility (K=10),test_utility (K=20)
SG n_sampling=1,19.6 ± 3.8,33.4 ± 5.7,57.5 ± 6.7
SG n_sampling=10,21.6 ± 3.5,36.6 ± 2.9,58.2 ± 7.2
SG n_sampling=100,26.3 ± 1.9,38.0 ± 4.5,61.8 ± 3.9
VR-SG n_sampling=10,27.0 ± 1.4,45.1 ± 1.2,71.4 ± 2.7
VR-SG n_sampling=100,27.6 ± 1.2,45.6 ± 1.9,73.9 ± 4.4
Continuous selector=sga,29.3 ± 1.8,45.7 ± 2.3,73.5 ± 3.6
Two-stage selector=greedy,11.0 ± 1.5,20.9 ± 1.6,41.4 ± 1.9
Random,11.9 ± 1.7,21.3 ± 2.1,38.8 ± 1.3


In [8]:
index = [key for key in full_results_table[MAX_CARDINALITIES[0]].keys()]
means = [
    [np.mean(val) for val in full_results_table[K].values()] for K in MAX_CARDINALITIES
]
stds = [
    [np.std(val) for val in full_results_table[K].values()] for K in MAX_CARDINALITIES
]
values = np.array(
    [
        [f"{m:.1f} ± {s:.1f}" for m, s in zip(means[i], stds[i])]
        for i in range(len(MAX_CARDINALITIES))
    ]
).T
print("Testing")
display(
    pd.DataFrame(
        values,
        index=index,
        columns=[
            f"test_utility (K={max_cardinality})"
            for max_cardinality in MAX_CARDINALITIES
        ],
    )
)

Testing


,test_utility (K=5),test_utility (K=10),test_utility (K=20)
SG n_sampling=1,23.1 ± 4.9,35.4 ± 5.5,61.6 ± 8.8
SG n_sampling=10,24.5 ± 2.4,41.7 ± 3.6,61.3 ± 9.7
SG n_sampling=100,28.0 ± 1.9,42.8 ± 6.8,66.5 ± 5.1
VR-SG n_sampling=10,30.2 ± 0.6,47.2 ± 2.4,76.8 ± 1.4
VR-SG n_sampling=100,28.6 ± 1.0,49.5 ± 1.0,79.0 ± 2.3
Continuous selector=sga,30.8 ± 1.3,48.0 ± 1.5,79.1 ± 1.1
Two-stage selector=greedy,11.2 ± 1.7,21.6 ± 1.4,41.5 ± 2.2
Random,10.4 ± 1.3,20.7 ± 0.7,41.2 ± 2.3
